<img src="https://github.com/hernancontigiani/ceia_memorias_especializacion/raw/master/Figures/logoFIUBA.jpg" width="500" align="center">


# Procesamiento de lenguaje natural
## RNN one-to-many

#### Datos
El objecto es utilizar una serie de sucuencias númericas (datos sintéticos) para poner a prueba el uso de las redes RNN. Este ejemplo se inspiró en otro artículo, lo tienen como referencia en el siguiente link:\
[LINK](https://stackabuse.com/solving-sequence-problems-with-lstm-in-keras-part-2/)

In [ ]:
import re

import numpy as np
import pandas as pd

from keras.preprocessing.text import one_hot
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers.core import Activation, Dropout, Dense
from keras.layers import Flatten, LSTM
from keras.layers import GlobalMaxPooling1D
from keras.models import Model
from keras.layers.embeddings import Embedding
from sklearn.model_selection import train_test_split
from keras.preprocessing.text import Tokenizer
from keras.layers import Input
from keras.layers.merge import Concatenate
from keras.layers import Bidirectional

In [ ]:
# Generar datos sintéticos
X = list()
y = list()

# X es una lista de números de 1 al 43 que avanzan de 3 en 3
X = [x for x in range(1, 44, 3)]

# "y" (target) se obtiene como por cada dato de entrada se
# se obtienen dos datos de salida como x+1 y x+2
y = [ [x+1, x+2] for x in X]

print("datos X:", X)
print("datos y:", y)

In [ ]:
# Cada dato X lo transformarmos en una matriz de 1 fila 1 columna (1x1)
X = np.array(X).reshape(len(X), 1, 1)
print("datos X:", X)

In [ ]:
y = np.asanyarray(y)
y.shape

### 2 - Entrenar el modelo

In [ ]:
input_shape = X[0].shape
input_shape

In [ ]:
output_shape = y.shape[1]
output_shape

In [ ]:
model = Sequential()
model.add(LSTM(64, activation='relu', input_shape=input_shape))
model.add(Dense(output_shape))
model.compile(loss='mse',
              optimizer="Adam")

model.summary()

In [ ]:
hist = model.fit(X, y, epochs=500, validation_split=0.2, batch_size=5)

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Entrenamiento
epoch_count = range(1, len(hist.history['loss']) + 1)
sns.lineplot(x=epoch_count,  y=hist.history['loss'], label='train')
sns.lineplot(x=epoch_count,  y=hist.history['val_loss'], label='valid')
plt.show()

In [ ]:
# Ensayo
x_test = 10
y_test = [x_test + 1, x_test + 2]
test_input = np.array([x_test])
test_input = test_input.reshape((1, 1, 1))
y_hat = model.predict(test_input, verbose=0)[0]

print("y_test:", y_test)
print("y_hat:", y_hat)

model.evaluate(test_input, np.array([y_test]))

### 3 - Multi-layer RNN

In [ ]:
# En esta oportunidad se utilizarán dos layer LSTM. Para poder conectar
# la primera layer con la segunda se debe colocar return_sequences=True

model2 = Sequential()
model2.add(LSTM(64, activation='relu', return_sequences=True, input_shape=input_shape))
model2.add(LSTM(64, activation='relu'))
model2.add(Dense(output_shape))
model2.compile(loss='mse',
              optimizer="Adam")

model2.summary()

In [ ]:
hist2 = model2.fit(X, y, epochs=500, validation_split=0.2, batch_size=5)

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Entrenamiento
epoch_count = range(1, len(hist2.history['loss']) + 1)
sns.lineplot(x=epoch_count,  y=hist2.history['loss'], label='train')
sns.lineplot(x=epoch_count,  y=hist2.history['val_loss'], label='valid')
plt.show()

In [ ]:
# Ensayo
x_test = 10
y_test = [x_test + 1, x_test + 2]
test_input = np.array([x_test])
test_input = test_input.reshape((1, 1, 1))
y_hat = model2.predict(test_input, verbose=0)[0]

print("y_test:", y_test)
print("y_hat:", y_hat)

model2.evaluate(test_input, np.array([y_test]))

### 4 - Conclusión
La unica diferencia que se debe tener en cuenta cuando hay más de una salida es que la cantidad de neuronas de la última capa debe coincidir con el tamaño de la secuencia de salida.
En este ejemplo, donde el problema es más complejo, hubo una diferencia apreciable entre utilizar una sola capa o varias LSTM.